# translate elec_loc.m to python

In [1]:
import nibabel as nib
from nibabel.gifti.giftiio import read
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def get_vertices(gs):
# get a matrix of vertices from a gii file
# gs: struct, nib.load(file)
# gs_vertices: array
    g = gs.darrays
    gs_vertices = g[0].data
    return  gs_vertices

In [3]:
def get_faces(gs):
# get a matrix of faces from a gii file
# values-- when loading, dunno why...
# gs: struct, nib.load(file)
# gs_faces: array
    g = gs.darrays
    gs_faces = g[1].data #+ 1
    return  gs_faces

In [4]:
def get_elec_pos(pos_txt):
# get an array of electrodes' positions
# pos_txt: string, file position
# pos_elec: array
    pos_elec = np.loadtxt(pos_txt, delimiter=',')
    return pos_elec

## get surface info

In [5]:
# nibabel load a patient's gii file
gs = nib.load('../image_data/yejiguo/surf/lh.central.yejiguoT1.gii')

# get a patient's arrays of vertices and faces
gls_vertices = get_vertices(gs)
gls_faces = get_faces(gs)

# get a patient's electrode position info
pos_txt = '../image_data/yejiguo/yejiguo.txt'
pos_elec = np.mat(get_elec_pos(pos_txt))
num_elec = pos_elec.shape[0]
#print(pos_elec)

### surface vertices

In [6]:
print(gls_vertices)

[[ -2.1936793  16.686512   11.022675 ]
 [-10.728247  -58.31927     7.5500326]
 [-46.966923   -4.4485855   2.194128 ]
 ...
 [ -9.084836  -92.472435   -7.890191 ]
 [ -6.9711304 -92.242874   -3.9177232]
 [ -6.591507  -92.1405     -5.015671 ]]


### surface faces

In [7]:
print(gls_faces)

[[    2 40962 10244]
 [40962 10242 10244]
 [10242  2562 10243]
 ...
 [38496 98303 30321]
 [98303 40961 30321]
 [38496 30321     1]]


In [8]:
print(pos_elec)

[[ 28.172     72.8509    42.8535    36.0242    17.7255   -20.9505  ]
 [ 59.7819    60.5681    12.9265    30.2571    40.6846    -4.03001 ]
 [ 63.5062    43.3856    12.768     31.6889    24.7548    -5.90958 ]
 [ 64.1499    53.4641    24.9266     1.07645   53.3077    10.3069  ]
 [ 46.8736    28.9822    42.2875    35.1313    15.9713   -15.5306  ]
 [ 28.6279    -0.218032  49.3625    34.6742     9.83906  -20.0579  ]
 [ 55.9013    10.3627    20.091      0.370398   3.48175   21.4089  ]
 [ 52.0515    -3.03456    5.46717   27.1005     2.50848   -4.79791 ]
 [  8.05622  -17.0832    49.9439    10.1689   -28.901    -12.059   ]]


## get electrodes info

In [9]:
# set up electrode array info
# electrode name
l_ename = ['A','B','C','D','E','F','G','H','J']
#r_ename = {}
# contact number on each electrode
l_elec_num = np.mat([16,14,14,16,16,16,16,8,16]) # list
#r_elec_num = []
# if electrode is separate
l_separate = np.mat([0,0,0,1,1,1,0,0,0]) # list
#r_separate = []

In [10]:
# calculate pall and enameloc
pin = pos_elec[:, 0:3]
pta = pos_elec[:, 3:6]
pvec = pin - pta
info_elec = {}
enameloc = {}
pall = {}
eistart = 0
preal = []

In [11]:
pvec_norm = pvec / np.linalg.norm(pvec, axis=1, keepdims=True)
for i in range(num_elec):
    temp_pvec = pvec_norm[i, :]
    tempp = np.zeros((l_elec_num[0, i], 4))
    if i > 0:
        eistart = eistart + l_elec_num[0, i-1]
    
    if l_separate[0, i] == 0:
        for j in range(l_elec_num[0, i]):
            tempp[j, 0:3] = pta[i, :] + (3.5*j - 1) * temp_pvec
    else:
        for j in range(8):
            tempp[j, 0:3] = pta[i, :] + (3.5*j - 1) * temp_pvec
        for j in range(8, 16):
            tempp[j, 0:3] = pta[i, :] + (3.5*j - 1) * temp_pvec + 8.5 * temp_pvec
    #print(tempp)
    enameloc[i] = tempp[j, 0:3] + 10 * temp_pvec
    tempp[:, 0] = - tempp[:, 0]
    tempp[:, 1] = - tempp[:, 1]
    pall[i] = tempp

info_elec['enameloc'] = enameloc
info_elec['pall'] = pall
#print(info_elec)

### electrodes name location

In [12]:
print(info_elec['enameloc'])

{0: matrix([[30.32171732, 57.75907513, 25.38571212]]), 1: matrix([[71.06793169, 68.168687  , 19.40822754]]), 2: matrix([[73.64333201, 49.32145499, 18.71875931]]), 3: matrix([[69.26837403, 53.47679202, 26.11300339]]), 4: matrix([[48.73631298, 31.04615445, 51.45934242]]), 5: matrix([[28.66269958, -0.16014824, 48.96294973]]), 6: matrix([[61.38670326, 11.04240777, 19.9608164 ]]), 7: matrix([[57.44724797, -4.23326333,  7.68703231]]), 8: matrix([[  8.11156535, -17.39278794,  48.3196256 ]])}


### electrodes location

In [13]:
print(info_elec['pall'])

{0: array([[-36.1169233 , -17.07454756, -21.70393434,   0.        ],
       [-35.79239176, -19.3528811 , -19.06691414,   0.        ],
       [-35.46786023, -21.63121465, -16.42989394,   0.        ],
       [-35.14332869, -23.90954819, -13.79287374,   0.        ],
       [-34.81879716, -26.18788173, -11.15585354,   0.        ],
       [-34.49426562, -28.46621528,  -8.51883333,   0.        ],
       [-34.16973409, -30.74454882,  -5.88181313,   0.        ],
       [-33.84520255, -33.02288237,  -3.24479293,   0.        ],
       [-33.52067102, -35.30121591,  -0.60777273,   0.        ],
       [-33.19613948, -37.57954945,   2.02924747,   0.        ],
       [-32.87160795, -39.857883  ,   4.66626768,   0.        ],
       [-32.54707642, -42.13621654,   7.30328788,   0.        ],
       [-32.22254488, -44.41455009,   9.94030808,   0.        ],
       [-31.89801335, -46.69288363,  12.57732828,   0.        ],
       [-31.57348181, -48.97121717,  15.21434848,   0.        ],
       [-31.24895028,

## plot

In [14]:
import mayavi
from mayavi import mlab
mlab.init_notebook(backend='ipy')

Notebook initialized with ipy backend.


In [15]:
cur_kwargs = dict(color = (0,0,0), scale_factor = 2.5, resolution=25)

for i in range(0, 9):
    mlab.points3d(info_elec['pall'][i][:,0], info_elec['pall'][i][:,1], info_elec['pall'][i][:,2], **cur_kwargs)
    #mlab.text3d(info_elec['enameloc'][i][:,0], info_elec['enameloc'][i][:,1], info_elec['enameloc'][i][:,2], l_ename[i], color=(0,0,0), orient_to_camera=False, line_width=5.0, scale=1.5)
mlab.triangular_mesh(gls_vertices[:, 0], gls_vertices[:, 1], gls_vertices[:, 2], gls_faces, color=(1,1,1), opacity=0.5)
mlab.triangular_mesh(-gls_vertices[:, 0], gls_vertices[:, 1], gls_vertices[:, 2], gls_faces, color=(1,1,1), opacity=0.5)
# mlab.show()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

In [16]:
# mlab.points3d(info_elec['pall'][0][:,0], info_elec['pall'][0][:,1], info_elec['pall'][0][:,2], **cur_kwargs)
# mlab.points3d(info_elec['pall'][0][:,0], info_elec['pall'][0][:,1], info_elec['pall'][0][:,2], **cur_kwargs)
# mlab.points3d(info_elec['pall'][1][:,0], info_elec['pall'][1][:,1], info_elec['pall'][1][:,2], **cur_kwargs)
# mlab.points3d(info_elec['pall'][2][:,0], info_elec['pall'][2][:,1], info_elec['pall'][2][:,2], **cur_kwargs)
# mlab.points3d(info_elec['pall'][3][:,0], info_elec['pall'][3][:,1], info_elec['pall'][3][:,2], **cur_kwargs)
# mlab.points3d(info_elec['pall'][4][:,0], info_elec['pall'][4][:,1], info_elec['pall'][4][:,2], **cur_kwargs)
# mlab.points3d(info_elec['pall'][5][:,0], info_elec['pall'][5][:,1], info_elec['pall'][5][:,2], **cur_kwargs)
# mlab.points3d(info_elec['pall'][6][:,0], info_elec['pall'][6][:,1], info_elec['pall'][6][:,2], **cur_kwargs)
# mlab.points3d(info_elec['pall'][7][:,0], info_elec['pall'][7][:,1], info_elec['pall'][7][:,2], **cur_kwargs)
# mlab.points3d(info_elec['pall'][8][:,0], info_elec['pall'][8][:,1], info_elec['pall'][8][:,2], **cur_kwargs)